# Create geojson file from list of PUDO coords
See /home/test/Documents/PROJECTS/VFH/iPython_Notebooks/dv_pudo_profiles.ipynb for more info.  

ref: https://geoffboeing.com/2015/10/exporting-python-data-geojson/  
Huom: `with open(output_filename, 'wb') as output_file:` should be `'w'` not `'wb'`  

In [1]:
import sys
sys.executable

'/home/test/anaconda3/envs/vfh/bin/python'

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import pandas.io.sql as pandasql
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as ticker

from IPython.display import HTML, display
import datetime
import matplotlib.dates as mdates
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import os

from datetime import datetime

import json

# Functions

In [3]:
# https://geoffboeing.com/2015/10/exporting-python-data-geojson/
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

# GeoJSON properties for file

In [4]:
# Define the properties of the PUDO geojson file
cols = ['type','dow','timewindow','latitude','longitude','intersection_id','counts']

# Variables

In [5]:
ward=1
this_day = "Monday"
this_tod = "amPeak"

# Read PUDO data from csv file

In [6]:
fname = '../../../Documents/PROJECTS/VFH/COT_PAGE/test_data/all_pudo_mon.csv'
# fname = '../../../Documents/PROJECTS/VFH/COT_PAGE/test_data/all_pudo_mon_ampeak.csv'
# fname = '~/Documents/PROJECTS/BDITTO/VFH/DATA_STORY/DATA/from_pgAdmin//w1_pudo_092018_latlon.csv'

df = pd.read_csv(fname)

print(df.shape)
df.head(5)

(494842, 14)


,pickup_datetime,dropoff_datetime,pickup_ward2018,dropoff_ward2018,pickup_gc_intersection_id,dropoff_gc_intersection_id,dow,timewindow,pickup_geom,dropoff_geom,lat_pickup,lon_pickup,lat_dropoff,lon_dropoff
0,2018-09-03 00:00:00,2018-09-03 00:34:01,18.0,10.0,13447475,13467654,Monday,nightII,0101000020E61000005BE2AF6670DA53C01D22DBEA93E2...,0101000020E610000013E6CE309BD953C0938FE7777DD2...,43.770139,-79.413110,43.644454,-79.400097
1,2018-09-03 00:00:00,2018-09-03 00:10:54,10.0,13.0,13467763,13464030,Monday,nightII,0101000020E610000051E519D30AD953C0C23B89A05BD2...,0101000020E610000079E6DB3E68D853C00F3AF08356D5...,43.643421,-79.391286,43.666703,-79.381363
2,2018-09-03 00:00:00,2018-09-03 00:12:34,13.0,10.0,13464551,13467763,Monday,nightII,0101000020E6100000A342F6F3C6D753C0E8EE4CFEEDD4...,0101000020E610000051E519D30AD953C0C23B89A05BD2...,43.663513,-79.371518,43.643421,-79.391286
3,2018-09-03 00:00:00,2018-09-03 00:24:18,13.0,3.0,13465140,13470547,Monday,nightII,0101000020E6100000DF5023ED39D853C020ED2E3976D4...,0101000020E610000004410BFFE5E153C041B447898CCC...,43.659858,-79.378536,43.598039,-79.529663
4,2018-09-03 00:00:00,2018-09-03 00:18:21,8.0,13.0,14044860,14673305,Monday,nightII,0101000020E6100000A37FC1EFB8DC53C070C432B4FCDB...,0101000020E61000008C84E11C68D853C062D98093A3D5...,43.718649,-79.448788,43.669054,-79.381355


In [7]:
list(df)

['pickup_datetime',
 'dropoff_datetime',
 'pickup_ward2018',
 'dropoff_ward2018',
 'pickup_gc_intersection_id',
 'dropoff_gc_intersection_id',
 'dow',
 'timewindow',
 'pickup_geom',
 'dropoff_geom',
 'lat_pickup',
 'lon_pickup',
 'lat_dropoff',
 'lon_dropoff']

In [8]:
df.loc[(df['pickup_gc_intersection_id']==13454935) & (df['dow']=='Monday') & (df['timewindow']=='amPeak')]

,pickup_datetime,dropoff_datetime,pickup_ward2018,dropoff_ward2018,pickup_gc_intersection_id,dropoff_gc_intersection_id,dow,timewindow,pickup_geom,dropoff_geom,lat_pickup,lon_pickup,lat_dropoff,lon_dropoff
333377,2018-09-10 08:00:00,2018-09-10 08:08:53,1.0,1.0,13454935,20232410,Monday,amPeak,0101000020E61000005851FD2F49E653C0139F29DE82DC...,0101000020E610000075DC3A49C4E653C09A9B9F7D4DDD...,43.722744,-79.598217,43.728927,-79.60573
350828,2018-09-10 08:00:00,2018-09-10 08:05:53,1.0,1.0,13454935,13459809,Monday,amPeak,0101000020E61000005851FD2F49E653C0139F29DE82DC...,0101000020E61000000DB97F50AAE553C0D1F8C0B9BFD8...,43.722744,-79.598217,43.693351,-79.58852


In [9]:
df.loc[(df['dropoff_gc_intersection_id']==13454935) & (df['dow']=='Monday') & (df['timewindow']=='amPeak')]

,pickup_datetime,dropoff_datetime,pickup_ward2018,dropoff_ward2018,pickup_gc_intersection_id,dropoff_gc_intersection_id,dow,timewindow,pickup_geom,dropoff_geom,lat_pickup,lon_pickup,lat_dropoff,lon_dropoff
428572,2018-09-03 08:00:00,2018-09-03 08:03:28,1.0,1.0,13455196,13454935,Monday,amPeak,0101000020E61000005B2EED6468E753C00EA1543F52DC...,0101000020E61000005851FD2F49E653C0139F29DE82DC...,43.72126,-79.615747,43.722744,-79.598217


In [10]:
df=

SyntaxError: invalid syntax (<ipython-input-10-02c2a1c0215a>, line 1)

# Housekeeping

## 1. Round lat, lon to 4 decimal places

In [ ]:
n=4 # number of decimals to round to

df['lat_pickup'] = df['lat_pickup'].round(decimals=n)
df['lon_pickup'] = df['lon_pickup'].round(decimals=n)

df['lat_dropoff'] = df['lat_dropoff'].round(decimals=n)
df['lon_dropoff'] = df['lon_dropoff'].round(decimals=n)

## 2. Filter out rows where neither pickup or dropoff wards are the ward of interest
Make col `keep` = 1 for where pick-up or for drop-off wards are the ward of interest

In [ ]:
# Assign a Pickup or Dropoff in column 'type'
df['keep'] = np.nan
df['latitude'] = np.nan
df['longitude'] = np.nan

df.loc[(df['pickup_ward2018']==ward) | (df['dropoff_ward2018']==ward), 'keep'] = 1

In [ ]:
# Check
df['keep'].unique()

In [ ]:
# Keep only rows where `keep` is not null
df = df[df['keep'].notnull()]

In [ ]:
# Check
df['keep'].unique()

In [ ]:
print(df.shape)

In [ ]:
df.head(5)

## 3a. Split df into two dfs for pickups and dropoffs separately
one where pickup ward == ward, the other where dropoff_ward == ward  

Assign `type` = 1 for pickup df, 5 for dropoff df.

In [ ]:
df_pu = pd.DataFrame()
df_pu = df.loc[df['pickup_ward2018']==ward]
df_pu['type'] = 1

In [ ]:
df_do = pd.DataFrame()
df_do = df.loc[df['dropoff_ward2018']==ward]
df_do['type'] = 5

In [ ]:
df_pu.head(4)

In [ ]:
# Fill `latitude` and `longitude` col with pickup or dropoff lat/lon
df_pu['latitude'] = df_pu['lat_pickup']
df_pu['longitude'] = df_pu['lon_pickup']

df_do['latitude'] = df_do['lat_dropoff']
df_do['longitude'] = df_do['lon_dropoff']

In [ ]:
df_pu.head(4)

In [ ]:
df_do.head(4)

## 3b. Aggregate pickups and dropoffs

In [ ]:
df_pu_agg=pd.DataFrame()

df_pu_agg = df_pu.groupby(
    ['type','dow','timewindow','latitude','longitude','pickup_gc_intersection_id']
    ).size().reset_index(name='counts')

df_pu_agg.head(3)

In [ ]:
# Check
df_pu_agg.loc[df_pu_agg['pickup_gc_intersection_id']==13454935]

In [ ]:
df_do_agg=pd.DataFrame()

df_do_agg = df_do.groupby(
    ['type','dow','timewindow','latitude','longitude','dropoff_gc_intersection_id']
    ).size().reset_index(name='counts')

df_do_agg.head(3)

In [ ]:
# Check
df_do_agg.loc[df_do_agg['dropoff_gc_intersection_id']==13454935]

## 4. Make a common table on common columns

### a. All points
Concat `df_pu` and `df_do`.

In [ ]:
# First rename intersection id to common name to allow concat
df_pu = df_pu.rename(columns={'pickup_gc_intersection_id': 'intersection_id'})
df_do = df_do.rename(columns={'dropoff_gc_intersection_id': 'intersection_id'})

In [ ]:
df_pudo=pd.DataFrame()

df_pudo=pd.concat([
    df_pu[['type','dow','timewindow','latitude','longitude','intersection_id']], 
    df_do[['type','dow','timewindow','latitude','longitude','intersection_id']]
    ])

df_pudo.head(4)

In [ ]:
# Check
df_pudo.loc[df_pudo['intersection_id']==13454935]

### a. Aggregates
Concat `df_pu_agg` and `df_do_agg`.

In [ ]:
# First rename intersection id to common name to allow concat
df_pu_agg = df_pu_agg.rename(columns={'pickup_gc_intersection_id': 'intersection_id'})
df_do_agg = df_do_agg.rename(columns={'dropoff_gc_intersection_id': 'intersection_id'})

In [ ]:
df_pudo_agg=pd.DataFrame()

df_pudo_agg=pd.concat([df_pu_agg[cols], df_do_agg[cols] ])

df_pudo_agg.head(4)

In [ ]:
# Check
df_pudo_agg.loc[df_pudo_agg['intersection_id']==13454935]

In [ ]:
df=

# Save to geojson

## All points

In [ ]:
geojson = df_to_geojson(df_pudo, ['dow', 'timewindow', 'type'])

## Aggregated points

In [ ]:
geojson_agg = df_to_geojson(df_pudo_agg, cols)

# Write geojson to file

## All points

In [ ]:
output_filename = '../src/geojson/w1_092018_' + this_day + '_' + this_tod + '.geojson'
# output_filename = 'w1_092018_' + this_day + '_' + this_tod + '.geojson'

with open(output_filename, 'w') as output_file:
    output_file.write('')
    json.dump(geojson, output_file, indent=2) 

## Aggregated points

In [ ]:
output_filename = '../src/geojson/w1_092018_' + this_day + '_' + this_tod + '_agg.geojson'
# output_filename = 'w1_092018_' + this_day + '_' + this_tod + '_agg.geojson'

with open(output_filename, 'w') as output_file:
    output_file.write('')
    json.dump(geojson_agg, output_file, indent=2) 

`
$ ls -lh src/geojson/
-rw-r--r-- 1 test test 328K Dec  4 15:46 w1_092018_Monday_amPeak_agg.geojson
-rw-r--r-- 1 test test 753K Dec  4 13:51 w1_092018_Monday_amPeak.geojson
`
